Written by Alexander Chkodrov. Modified by River Liu.

# (1) Data Structure

An HDF5 data container is a standardized, highly-customizable data receptacle designed for portability.


## Accessing the Data: Part 1

Navigate to the directory where you've saved the processed-pythia.z data file.

In [ ]:
import h5py

In [ ]:
f = h5py.File('data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z', 'r')

In [ ]:
f.keys()

In [ ]:
treeArray = f['t_allpar_new'][()] #Empty tuple indexing retrieves all values
print(treeArray.dtype.names)

## Understanding the Data
The structure of a jet is introduced in tutorial 12 from the computing tutorial. It is very important to understand the structure of the sample in order to see the applications of different network types. 

In our pre-processed sample, each row is a jet with the above features. The two and four layer networks use the features of the jet to classify whether the jet originated from the decay a top quark or not.

The features/inputs we will use are



|    Features    |    Labels     |
|  :-------:     |  :---------:  |
|  j_zlogz       |  j_t          |
|  j_c1_b0_mmdt        |               |
|  j_c1_b1_mmdt        |               |
|  j_c2_b1_mmdt        |               |
|  j_d2_b1_mmdt        |               |
|  j_d2_a1_b1_mmdt        |               |
|  j_m2_b1_mmdt        |               |
|  j_n2_b1_mmdt        |               |
|  j_mass_mmdt        |               |
|  j_multiplicity       |               |

And clearly our label will be top quark or not. A training sample contains both features and labels.

Not all networks use the already-clustered jets' features to classify the elementary particles from which they originated. The best performing ones are usually either _constituent_ or _image_ based classifiers. 

[ResNet-50](https://arxiv.org/pdf/1512.03385.pdf) is an example of an image-based classifier which achieves state of the art performance, with an input layer populated by the pixels in a _jet image_. The sensors on the inside surface of the cylindrical detectors can be represented in a two dimensional histogram of $\eta$ and $\phi$, and activation of these pixels is the energy or transverse momentum transferred to the sensors. This decision influences the architecture of the network as well as its performance; it requires immense amounts of data and computational power to train, as training time is proportional to the number of neurons a network needs to train (and with a 244x244 pixel image, thats an input layer of nearly 60,000 neurons). 

[Long Short-Term Memory](https://arxiv.org/pdf/1711.09059.pdf) (More [here](https://www.sciencedirect.com/science/article/pii/S0167278919305974)) is an example of a network which excells at consituent-based classification. The jets in our data sample come pre-clustered and pre-processed; with constituent based classifiers, the particles composing the jet itself are each individually analyzed. Constituent-based classification makes sense for a recurring neural network like LSTM, as the last constituent ought to influence the classification of the next constituent of the same jet. With a relatively small list of inputs and a simple network architecture, the LSTM network achieves very good performance with O(1000) neurons making it very quick to train and requires comparitively fewer data points to train.

## Accessing the Data: Part 2

In [ ]:
features = ['j_zlogz', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c2_b1_mmdt', 'j_d2_b1_mmdt', 'j_d2_a1_b1_mmdt',
            'j_m2_b1_mmdt', 'j_n2_b1_mmdt', 'j_mass_mmdt', 'j_multiplicity']
labels = ['j_t']

In [ ]:
features+labels

In [ ]:
import pandas as pd

In [ ]:
features_labels_df = pd.DataFrame(treeArray,columns=features+labels)
features_labels_df = features_labels_df.drop_duplicates()

In [ ]:
features_labels_df

The above dataframe represents ~1 million jets each with the listed features and labelled as top (1) or not (0).

One of the most important steps to ensure the robust-ness of your machine learning solution is to retain a portion of data as a testing set. Understand the testing set's importance [here](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7). It is also imperative to shuffle the data before training a neural network to reach the global minimum of loss as opposed to getting stuck at a local minimum. When trying to create reproducible results, it is also useful to specify a seed for the random number generators.

scikit-learn can seperate training and testing sets as well as shuffle the data with the useful method train_test_split.

In [ ]:
features_val = features_labels_df[features].values #Convert to numpy array
labels_val = features_labels_df[labels].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

Finally, we have shuffled training and testing data to use with our model. Now, follow the next tutorial to learn how to build a model.

##### Exercise

The four-layer model is more generally applicable due to its depth; instead of only tagging jets that originate from top quarks, it can tag jets originating from several different fundamental particles. Extract the training and testing data sets from the sample for the four-layer model training. The features and labels you are trying to extract are:


| Features | Labels |
|  :---:   |  :--:  |
j_zlogz  | j_g 
j_c1_b0_mmdt | j_q 
j_c1_b1_mmdt | j_w 
j_c1_b2_mmdt | j_z
j_c2_b1_mmdt | j_t
j_c2_b2_mmdt 
j_d2_b1_mmdt 
j_d2_b2_mmdt 
j_d2_a1_b1_mmdt 
j_d2_a1_b2_mmdt 
j_m2_b1_mmdt 
j_m2_b2_mmdt 
j_n2_b1_mmdt 
j_n2_b2_mmdt 
j_mass_mmdt 
j_multiplicity 

In [ ]:
# TODO


# (2) Creating a Model

The open source packages tensorflow and keras do most of the heavy lifting when it comes to machine learning computation. No need to worry about calculus and backpropogation, just build the model and use .fit(). Learn more about the types of models and a little bit about the math behind the keras and tensorflow functions from [UW CSE 416 Spring 2019](https://courses.cs.washington.edu/courses/cse416/19sp/lectures.html) or similiar in content but with complete access [Andrew Ng's Machine Learning Coursera](https://www.coursera.org/learn/machine-learning?utm_source=gg&utm_medium=sem&utm_content=07-StanfordML-US&campaignid=685340575&adgroupid=52515609594&device=c&keyword=machine%20learning%20mooc&matchtype=b&network=g&devicemodel=&adpostion=&creativeid=273169971757&hide_mobile_promo&gclid=Cj0KCQjwuJz3BRDTARIsAMg-HxX7mT2U1X1Abs98BkFp_IKCypGKMbWTjIiwx4GY-C-3LrQ5R82TtrkaAqn4EALw_wcB).

Now, for building the two-layer model.

In [ ]:
import tensorflow as tf

In [ ]:
# Keras documentation highly recommend we use tensorflow.keras instead of keras, so we will use tensorflow.keras in this case.
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

The input of the two-layer network includes 10 features,

In [ ]:
Inputs = Input(shape=(10,))

Arbitrarily we choose 32 as the number of neurons in our one hidden layer. This is a hyperparameter that is best optimized through experimentation, and largely dependent on the data set / function the neural network is approximating. [Source](https://machinelearningmastery.com/how-to-configure-the-number-of-layers-and-nodes-in-a-neural-network/).

When you know the function you are trying to approximate has certain characteristics, you can choose an activation function which will approximate the function faster leading to faster training process. ReLu is a good general approximator and Sigmoid is good for classifiers. [Source](https://medium.com/the-theory-of-everything/understanding-activation-functions-in-neural-networks-9491262884e0)

Kernel initialization is an important part of building deep neural networks. Proper initialization will help to extract better features and also to converge faster. So, we have to carefully select our filter initializers, but selecting the kernel initializers is also a kind of hyperparameter tuning (We can't tell exactly which kernel initializers to choose). It all depends on the nature of the dataset and the kind of operation you are going to perform on it. [Source](https://blog.goodaudience.com/visualizing-various-filter-initializers-in-keras-ca14c996db22).

For our hidden layer we will use ReLu activation, which is often a good place to start. The output layer will have sigmoid activation, which approximates the ideal classifier function. The initializer is the LeCun uniform initializer. It draws samples from a uniform distribution within [-limit, limit] where the limit is sqrt(3 / N) where N is the number of input units in the weight tensor. 

In [ ]:
x = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name='fc1_relu')(Inputs)

The output layer of our two-layer network is a single label, so there is 1 node.

In [ ]:
predictions = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'output_sigmoid')(x)

Read the [Keras](https://keras.io/api/layers/core_layers/dense/) and [Tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/lecun_uniform) documentation to understand the syntax better. Tensorflow 2 does the building of a computational graph and execution all in the backend; all thats left is the creation of these connected layers and putting them together in a Model object.

The model with one input layer, one hidden layer and one output layer is called a 'two-layer' model because the inputs are not considered an 'active' layer. the four-layer model similiarly has three hidden layers and one output layer.

In [ ]:
model = Model(inputs=Inputs, outputs=predictions)

The two-layer model is complete!

In [ ]:
model.summary()

Now all thats left is to use the model.fit() method to train it, validate the training, and evaluate its performance.

##### Exercise

Create the four-layer model. Use 64, 32, and 32 nodes for the first, second, and third hidden layers respectively. For all the hidden layers, use the ReLu activation function and LeCun uniform kernel initializer. Print the summary. (Hint: What's different about the output layer?)

In [ ]:
# TODO


# (3) Training and Validation

One of the key factors which limits the use of neural networks in many industrial applications has been the difficulty of demonstrating that a trained network will continue to generate reliable outputs once it is in routine use. [Source](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/05/Bishop-Novelty-Detection_IEE-Proceedings-94b.pdf).

That is the motivation for the [validation and test splits](https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7) mentioned in the (1) Data Structure tutorial. In that tutorial we seperated training and testing data, and later during the training we will specify the validation fraction. In order to train our model, first we need to extract the training and testing data and build the model.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import h5py
import pandas as pd

In [ ]:
Inputs = Input(shape=(10,))
x = Dense(32, activation='relu', kernel_initializer='lecun_uniform', name='fc1_relu')(Inputs)
predictions = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'output_sigmoid')(x)
model = Model(inputs=Inputs, outputs=predictions)

In [ ]:
f = h5py.File('data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z', 'r')
treeArray = f['t_allpar_new'][()]

features = ['j_zlogz', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c2_b1_mmdt', 'j_d2_b1_mmdt', 'j_d2_a1_b1_mmdt',
            'j_m2_b1_mmdt', 'j_n2_b1_mmdt', 'j_mass_mmdt', 'j_multiplicity']
labels = ['j_t']

features_labels_df = pd.DataFrame(treeArray,columns=list(set(features+labels)))
features_labels_df = features_labels_df.drop_duplicates()

features_val = features_labels_df[features].values #Convert to numpy array
labels_val = features_labels_df[labels].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_val, labels_val, test_size=0.2, random_state=42)

## Training

The [loss function](https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/) essentially represents the error calculated from predicted and true values; in practice, this function be hard to determine.

Binary cross-entropy is the standard loss function for binary classification problems. It is used when the target values are eitiher {0,1} and only one target is possible at a time. Mathematically, it is the preferred loss function under the inference framework of maximum likelihood. It is the loss function to be evaluated first and only changed if you have a good reason.

An optimizer is the function which updates the weight parameters to minimize the loss function. The loss function represents the surface, and the optimizer is the method to reach the lowest point of that surface. Here is an article about different kinds of [optimizers](https://medium.com/datadriveninvestor/overview-of-different-optimizers-for-neural-networks-e0ed119440c3). The [learning rate](https://towardsdatascience.com/understanding-learning-rates-and-how-it-improves-performance-in-deep-learning-d0d4059c1c10) affects the magnitude of change between each training example, or how 'far' to descend the loss gradient. It is generally a very small number, and there are callback methods to adjust the learning rate dynamically.

In this model, we will be using Adam because it is a generally effective optimizer.

In [ ]:
adam = Adam(lr=0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

Next, we will use the [model.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) function to train the network. Read about the parameters in the documentation.

Batch_size represents the size of data bins used to train the network, since with large volumes of data it cannot fit all onto your RAM at one time. An epoch is one iteration through the entire shuffled data set; with additional epochs, the data is reshuffled and used to train the network again. Callbacks are used in between epochs, and can be highly customized.

The validation split represents the fraction of the remaining training data to use as a validation set during the training. That is important so that the loss or error of the network will be computed on data which it hasn't trained on, leading to more robust solutions. Then towards the end, the network is again evaluated on the _test_ data set that we split off in the beginning, which the network has never seen before.

In [ ]:
history = model.fit(X_train, y_train, batch_size = 1024, epochs = 100, 
                    validation_split = 0.25, shuffle = True, callbacks = None,
                    use_multiprocessing=True, workers=4)

The model is now trained! It took 100s on my intel-i7 CPU. It is possible for tensorflow to be optimized using a GPU, if you have an NVIDIA graphics card that is CUDA compatible. Feel free to enable this yourself if applicable, but for these tutorials and many smaller models a CPU will work at a reasonable enough pace.

The output of the fit function is a history object, which contains the values of loss for the training data as well as the validation data, and whatever metrics specified for each epoch.

To save the model, Tensorflow has very user friendly functions,

In [ ]:
model.save('two-layer.h5') #Saves to local h5 file

And it is just as easy to load the model. This is useful when the training takes a long time, so you only need to train it once. Both the save and load arguments can take different paths to the file, but here it is in the local directory.

In [ ]:
loaded_model = tf.keras.models.load_model('two-layer.h5') #Loads from local h5 file

In [ ]:
loaded_model.summary()

## Validation

To validate the robustness of the networks predictions and ensure that the network isn't overfit on our sample, we want to compare the loss on the training set versus the loss on the validation set. The plot comparing the two over each epoch is called the 'learning curve'.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def learningCurve(history):
    plt.figure()
    plt.plot(history.history['loss'], linewidth=1)
    plt.plot(history.history['val_loss'], linewidth=1)
    plt.title('Model Loss over Epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['training sample loss','validation sample loss'])
    plt.show()
    plt.close()

In [ ]:
learningCurve(history)

Divergence of the validation and training sample losses implies overfitting on your training data set. The above is quite a beautiful learning curve. You can see that it converges around 0.3, and every training epoch after around 10 epochs gives marginal improvement.

So we've discovered that the model is not overfit, next we want to know how it performs. One very popular measure of _classifier_ performance is called the [receiver operating characteristic (ROC) curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). It is a plot of the true positive rate (signal efficiency) versus the false positive rate (background efficiency). [Understanding the ROC curve](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5).

We will use our *test* sample that we split in the very beginning, and use our network to predict the output and compare it with the true output.

In [ ]:
labels_pred = model.predict(features_val)

That was the _inference_ calculation. The network just predicted the labels based on the features provided. I am providing you with this method to plot the learning curve, as this is a machine learning tutorial not a plotting tutorial; that said, you should inspect the code and make sure you understand the use of roc_curve() and auc() and what exactly is being plotted.

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
def makeRoc(features_val, labels_val, labels, model, outputDir='', outputSuffix=''):
    labels_pred = model.predict(features_val)
    df = pd.DataFrame()
    fpr = {}
    tpr = {}
    auc1 = {}
    plt.figure()       
    for i, label in enumerate(labels):
        df[label] = labels_val[:,i]
        df[label + '_pred'] = labels_pred[:,i]
        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])
        auc1[label] = auc(fpr[label], tpr[label])
        plt.plot(fpr[label],tpr[label],label='%s tagger, AUC = %.1f%%'%(label.replace('j_',''),auc1[label]*100.))
    plt.xlabel("Background Efficiency")
    plt.ylabel("Signal Efficiency")
    plt.xlim([-0.05, 1.05])
    plt.ylim(0.001,1.05)
    plt.grid(True)
    plt.legend(loc='lower right')
    plt.title('%s ROC Curve'%(outputSuffix))
    #plt.savefig('%s_ROC_Curve.png'%(outputSuffix))
    return labels_pred

In [ ]:
y_pred = makeRoc(X_test, y_test, labels, model, outputSuffix='two-layer')

The above is the learning curve for the two-layer model. A perfect classifier would look like a line from coordinates (0,0) to (1,0) to (1,1); following the left and upper border of the plot. A classifier whose AUC is below 50% can be reversed in implementation to give greater than 50% AUC. The worst possible classifier would follow the black dotted line, with a minimum AUC of 50% and offer no greater descrimination than flipping a coin.

After the exercise, this concludes the DNN Tutorial. You've downloaded and understood the structure of a sample LHC data set (pre-processed; the processing of the data is another problem of its own), built a model, trained it, and validated it through inspecting the learning curve and ROC curve. Congratulations!

##### Exercise

Train and plot the learning and ROC curves of the four-layer model. From (1)Data Structure **exercise** we have already extract dataset we need for this model. Use the Adam and categorical-crossentropy losses with the same parameters. The training might take longer, so it is okay to stop it once it has more or less converged ( >= 20 epochs). Use the provided methods to do the plotting.

In [ ]:
# TODO
